In [1]:
import tensorflow as tf
import os
from models.sentencecnn import SentenceCNN
from models.gradcam import GradCAM
from tqdm import trange
import numpy as np

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

c:\users\max\edu\kth\dd2412\project\src\venv\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\max\edu\kth\dd2412\project\src\venv\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\max\edu\kth\dd2412\project\src\venv\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


1 Physical GPUs, 1 Logical GPUs


In [2]:
cnn = SentenceCNN("dataset-text/train.tsv/train.tsv", "dataset-text/glove.6B.200d.txt")
cnn.load_data()
# cnn.initalize_model()
# cnn.train("checkpoints")
cnn.load_model("checkpoints_1")

Loaded 400000 word vectors.


In [3]:
cnn.model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 52)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 52, 200)      2804600     input_1[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 52, 200, 1)   0           embedding[0][0]                  
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 50, 1, 100)   60100       reshape[0][0]                    
_______________________________________________________________________________________

In [16]:
index = 148#5942
print(cnn.sst2_data["Phrase"].values[index])
print(cnn.sst2_X[index])
print(cnn.sst2_y[index])
c = np.argmax(cnn.sst2_y[index])
print(c)

depressing to see how far Herzog has fallen
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0  688    6  106  113  180 1717   34 4601]
tf.Tensor([1. 0.], shape=(2,), dtype=float32)
0


We see that index('is') = 7

In [5]:
cnn.sst2_word_index['boasts'] # WOHO

4033

In [6]:
reverse_dict = {}
for key in cnn.sst2_word_index.keys():
    reverse_dict[cnn.sst2_word_index[key]] = key

In [7]:
reverse_dict[441]

'having'

In [17]:
gradcam = GradCAM(cnn.model)
c = np.argmax(cnn.model.predict(tf.expand_dims(cnn.sst2_X[index], 0)))
print(c)
heatmap1 = gradcam.get_heatmap_text(c, tf.expand_dims(cnn.sst2_X[index], 0), "conv2d", inverse=True)
heatmap2 = gradcam.get_heatmap_text(c, tf.expand_dims(cnn.sst2_X[index], 0), "conv2d_1", inverse=True)
heatmap3 = gradcam.get_heatmap_text(c, tf.expand_dims(cnn.sst2_X[index], 0), "conv2d_2", inverse=True)
#heatmap_avg = np.mean(np.stack([heatmap1, heatmap2, heatmap3], axis=-1), -1)

0


In [18]:
cnn.model.predict(tf.expand_dims(cnn.sst2_X[index], 0))

array([[0.9845084 , 0.01028011]], dtype=float32)

In [22]:
heatmap_list = heatmap2.T[0]
for i in range(len(heatmap_list)):
    word_index = cnn.sst2_X[index][i]
    if word_index != 0:
        print(reverse_dict[word_index], heatmap_list[i])

depressing 0.19941427
to 0.008567826
see 0.014312509
how 0.0
far 0.21674179
herzog 1.0
has 0.0
fallen 0.49006057
